In [ ]:
import pandas as pd

df_PS = pd.read_csv("csv/PS_civils.csv", sep=";", dtype=str)
df_communes = pd.read_csv("csv/communes.csv", sep=";", dtype=str)

In [ ]:
df_communes['population']=df_communes['population'].astype('int64')

CLASSIFICATION DES COMMUNES SELON LEUR POPULATION

In [ ]:
def get_code_pop(p):
    if p<2000:
        return '1 - commune'
    elif (p<5000):
        return '2 - bourg'
    elif (p<20000):
        return "3 - petite ville"
    elif (p<50000):
        return '4 - ville moyenne'
    elif (p<200000):
        return '5 - grande ville'
    else :
        return '6 - metropole'

In [ ]:
df_communes['taille_commune']=df_communes['population'].apply(lambda x : get_code_pop(x))

POPULATION PAR DEPARTEMENT, PAR REGION


In [ ]:
df_population_par_ville = df_communes.groupby(['code_insee', 'nom_standard'])['population'].sum().reset_index()
df_population_par_dep   = df_communes.groupby(['dep_code','dep_nom'])["population"].sum().reset_index()

NOMBRE DE PS PAR DEPARTEMENT ET PAR REGION

In [ ]:
df_PS_communes = df_PS.merge(df_communes,left_on=['Code commune (coord. structure)'], right_on=['code_insee'])

df_PS_par_dep= df_PS_communes.groupby(['dep_code','dep_nom', 'Libellé profession','Libellé savoir-faire','Libellé civilité','Libellé mode exercice'],dropna=False)["Identifiant PP"].count().reset_index()
df_PS_par_com= df_PS_communes.groupby(['Code commune (coord. structure)','Libellé profession','Libellé savoir-faire','Libellé civilité','Libellé mode exercice'],dropna=False)["Identifiant PP"].count().reset_index()

In [ ]:
df_PS_par_dep_avec_pop = df_PS_par_dep.merge(df_population_par_dep,on=['dep_code'])
df_PS_par_com_avec_pop = df_PS_par_com.merge(df_population_par_ville,left_on=['Code commune (coord. structure)'], right_on=['code_insee'])

CALCUL DU RATIO POUR 1000 HABITANTS

In [ ]:
df_PS_par_dep_avec_pop['population']=df_PS_par_dep_avec_pop['population'].astype('int64')

In [ ]:
df_PS_par_dep_avec_pop['ratio']=df_PS_par_dep_avec_pop.apply(lambda x: x["Identifiant PP"]/x["population"]*1000, axis=1)


In [ ]:
df_PS_par_dep_avec_pop=df_PS_par_dep_avec_pop.drop(columns=['dep_nom_y'])

EXPORT CSV POUR LA SUITE

In [ ]:
df_PS_par_dep_avec_pop.to_csv('csv/df_PS_par_dep_avec_pop.csv', sep=";",index=False)
df_PS_par_com_avec_pop.to_csv('csv/df_PS_par_com_avec_pop.csv', sep=";",index=False)